## Final Project Submission

* Student name: Yi-Wei Liu
* Student pace: Flex
* Scheduled project review date/time: Nov 16, 2023 (10:00-10:45 AM)
* Instructor name: Morgan Jones
* Blog post URL: https://medium.com/@yw.liu93/data-science-and-efficient-financial-markets-efc812877fdb

###  Introduction
In this project, a hypothetical company (the "Company") is expanding into the business of operating aircraft for enterprises. The company would like to identify aircraft with the best safety record. For the purposes of this project, I define "best safety record" as having the lowest ratio of accidents involving fatal injuries to total number of accidents. 

This project will analyze the safety record of each major manufacturer, with further breakdowns by flight location, aircraft purpose and weather conditions. With the results of this project, the Company would be better informed on the long-term safety record of different manufacturers when negotiating with potential vendors. The Company would also have better insight on the risk of poor weather on flight safety.

###  Data Source
This project's data source is a dataset from the National Transportation Safety Board ("NTSB Dataset"). The NTSB Dataset includes information on 90,348 aviation accidents from 1962 to 2023 in the U.S. and in international waters, with details on 30 variables.  

Link to NTSB Dataset: https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses

#### Suitability of the NTSB Dataset for this project
(+) Spans a long period of time, able to assess manufacturers' long-term record  
(+) Relatively well-maintained data for variables relevant to our business questions (manufacturer, location, purpose, weather, number of fatal injuries)

(-) Does not include any data or metadata on flights not involved in accidents; as a result, we are unable to assess the accident rate as a percentage of all flights. Organizations like IATA typically report aircraft safety in terms of number of total or fatal accidents per million passenger miles.

### Business Questions and Project Focus
For this project, I will focus only on the aircraft manufacturer and not on specific aircraft models. The Company is seeking a long-term vendor to be a reliable supplier across a range of model types.

I will also answer the following business questions:

1) Location: Does the safety record vary between flights in the United States vs. overseas?

2) Purpose: Does the safety record vary between aircraft for personal and commercial use?

3) Weather: Is bad weather a major factor in causing fatal accidents?

### Data Preparation
In the below section, I clean up the data to more efficiently answer the above business questions.

In [2]:
import pandas as pd
import numpy as np

# We start with a general look at the dataset:
aircraft_df = pd.read_csv('data/Aviation_Data.csv', low_memory=False)
aircraft_df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


In [2]:
aircraft_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,10/24/1948,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,7/19/1962,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,8/30/1974,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,6/19/1977,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12/9/2000
4,20041105X01764,Accident,CHI79FA064,8/2/1979,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [2]:
# There are many columns not relevant to our analysis. 
# We'll keep the date, country, make (manufacturer), purpose, total fatal injuries and weather columns.
analysis_df = pd.read_csv('data/Aviation_Data.csv', usecols=[3,5,14,21,23,27])
analysis_df.head()

,Event.Date,Country,Make,Purpose.of.flight,Total.Fatal.Injuries,Weather.Condition
0,10/24/1948,United States,Stinson,Personal,2.0,UNK
1,7/19/1962,United States,Piper,Personal,4.0,UNK
2,8/30/1974,United States,Cessna,Personal,3.0,IMC
3,6/19/1977,United States,Rockwell,Personal,2.0,IMC
4,8/2/1979,United States,Cessna,Personal,1.0,VMC


In [3]:
# The data goes as far back as 1948. To keep the analysis relevant to today, we set a cut-off date of Jan 1, 2000:
analysis_df['Event.Date'] = pd.to_datetime(analysis_df['Event.Date'])
filtered_df = analysis_df[(analysis_df['Event.Date'] >= '2000-01-01')]
filtered_df.head()

,Event.Date,Country,Make,Purpose.of.flight,Total.Fatal.Injuries,Weather.Condition
47675,2000-01-01,United States,Cessna,Positioning,0.0,VMC
47676,2000-01-01,United States,Bellanca,Personal,2.0,IMC
47677,2000-01-02,United States,Cessna,Personal,0.0,VMC
47678,2000-01-02,United States,Cessna,Instructional,0.0,VMC
47679,2000-01-02,United States,Piper,Personal,0.0,VMC


In [4]:
# Let's clean up the Make / Manufacturer column: 
filtered_df['Make'].value_counts()

Cessna               5666
CESSNA               4922
Piper                3075
PIPER                2841
Beech                1212
                     ... 
Kenneth Parmerter       1
Bacon                   1
Betts                   1
CAMPBELL JOHN           1
Steinke                 1
Name: Make, Length: 5811, dtype: int64

In [5]:
# The capitalization of each manufacturer's name should be standardized:
filtered_df['Make'] = filtered_df['Make'].str.title()
filtered_df['Make'].value_counts()

<ipython-input-5-dc9cd1cefda8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Make'] = filtered_df['Make'].str.title()


Cessna                   10588
Piper                     5916
Beech                     2254
Boeing                    1741
Bell                      1216
                         ...  
Barber Warren D              1
Western International        1
Christopher John P           1
Ronald J Bender              1
Steinke                      1
Name: Make, Length: 5245, dtype: int64

In [6]:
# The long tail of names with just one value will take a long time to individually check and clean up.
# To simplify the task, I eliminate the names which appear less than 100 times:
value_counts = filtered_df['Make'].value_counts()
to_keep = value_counts[value_counts > 100].index
filtered_df = filtered_df[filtered_df['Make'].isin(to_keep)]

filtered_df['Make'].value_counts()

Cessna                         10588
Piper                           5916
Beech                           2254
Boeing                          1741
Bell                            1216
Robinson                         808
Mooney                           580
Air Tractor                      364
Hughes                           348
Bellanca                         338
Schweizer                        336
Maule                            293
Airbus                           291
Aeronca                          272
Mcdonnell Douglas                238
Grumman                          232
Robinson Helicopter              230
Cirrus Design Corp               223
Air Tractor Inc                  219
Champion                         200
Eurocopter                       198
Embraer                          195
Robinson Helicopter Company      194
Luscombe                         192
Stinson                          185
North American                   174
De Havilland                     158
A

In [7]:
# Now, it's easier to spot which manufacturers' names need to be standardized:
filtered_df['Make'].replace(['Air Tractor Inc','Cirrus Design Corp', 'Dehavilland',
                             'Robinson Helicopter Company','Robinson Helicopter'], 
                            ['Air Tractor','Cirrus','De Havilland',
                             'Robinson','Robinson'], inplace=True)
filtered_df['Make'].value_counts()

Cessna               10588
Piper                 5916
Beech                 2254
Boeing                1741
Robinson              1232
Bell                  1216
Air Tractor            583
Mooney                 580
Cirrus                 371
Hughes                 348
Bellanca               338
Schweizer              336
Maule                  293
Airbus                 291
Aeronca                272
De Havilland           259
Mcdonnell Douglas      238
Grumman                232
Champion               200
Eurocopter             198
Embraer                195
Luscombe               192
Stinson                185
North American         174
Aero Commander         149
Taylorcraft            140
Sikorsky               132
Aerospatiale           127
Vans                   125
Aviat                  118
Enstrom                107
Name: Make, dtype: int64

In [8]:
# The shortlist of 31 manufacturers is still too long for the company to screen.
# We'll focus on the top 25% of manufacturers, i.e. 8 largest manufacturers:
value_counts = filtered_df['Make'].value_counts()
to_keep = value_counts[value_counts >= 580].index
filtered_df = filtered_df[filtered_df['Make'].isin(to_keep)]

filtered_df['Make'].value_counts()

Cessna         10588
Piper           5916
Beech           2254
Boeing          1741
Robinson        1232
Bell            1216
Air Tractor      583
Mooney           580
Name: Make, dtype: int64

In [9]:
# Next, let's clean up the 'Country' column:
filtered_df['Country'].value_counts()

United States                       20535
Brazil                                255
Mexico                                200
Canada                                198
Australia                             197
                                    ...  
Cyprus                                  1
Solomon Islands                         1
United Arab Emirates                    1
Guinea                                  1
Saint Vincent and the Grenadines        1
Name: Country, Length: 182, dtype: int64

In [10]:
# We'll directly address the business question of U.S. vs. overseas by modifying the name of non-US countries to 'Overseas':
filtered_df.loc[filtered_df['Country'] != 'United States', 'Country'] = 'Overseas'

# We'll also rename the column from 'Country' to 'Location'
filtered_df.rename(columns={'Country': 'Location'}, inplace=True)
filtered_df['Location'].value_counts()

United States    20535
Overseas          3575
Name: Location, dtype: int64

In [11]:
# Next, let's clean up the Purpose column:
filtered_df.rename(columns={'Purpose.of.flight': 'Purpose'}, inplace=True)
filtered_df['Purpose'].value_counts()

Personal                     12605
Instructional                 3573
Aerial Application             990
Business                       700
Unknown                        558
Positioning                    498
Other Work Use                 321
Aerial Observation             280
Public Aircraft                139
Skydiving                      130
Ferry                          117
Flight Test                    115
Executive/corporate            107
Banner Tow                      78
Public Aircraft - State         44
Public Aircraft - Local         43
External Load                   36
Public Aircraft - Federal       35
Glider Tow                      28
Firefighting                    22
Air Race show                    6
Air Drop                         5
Air Race/show                    4
ASHO                             3
PUBS                             3
PUBL                             1
Name: Purpose, dtype: int64

In [12]:
# We'll eliminate rows with long-tail purposes not relevant to our analysis, such as 'Unknown' and 'Skydiving':
irrelevant = ['Unknown','Skydiving','Flight Test',
              'Glider Tow','Firefighting','Air Race show',
              'Air Drop','Air Race/show','PUBS','ASHO','PUBL']

filtered_df.drop(filtered_df[filtered_df['Purpose'].isin(irrelevant)]
                        .index, inplace=True)

filtered_df['Purpose'].value_counts() 

Personal                     12605
Instructional                 3573
Aerial Application             990
Business                       700
Positioning                    498
Other Work Use                 321
Aerial Observation             280
Public Aircraft                139
Ferry                          117
Executive/corporate            107
Banner Tow                      78
Public Aircraft - State         44
Public Aircraft - Local         43
External Load                   36
Public Aircraft - Federal       35
Name: Purpose, dtype: int64

In [13]:
# We'll group 'Instructional' flights with 'Personal', and group the remaining categories into 'Commercial & Public'
filtered_df.loc[filtered_df['Purpose'] == 'Instructional', 'Purpose'] = 'Personal'
filtered_df.loc[filtered_df['Purpose'] != 'Personal', 'Purpose'] = 'Commercial & Public'
filtered_df['Purpose'].value_counts() 

Personal               16178
Commercial & Public     7057
Name: Purpose, dtype: int64

In [14]:
# Finally, let's clean up the Weather column:
filtered_df.rename(columns={'Weather.Condition': 'Weather'}, inplace=True)
filtered_df['Weather'].value_counts()

VMC    18981
IMC     1436
Unk      155
UNK       15
Name: Weather, dtype: int64

In [15]:
# We'll standardize "UNK" and replace these abbreivations with easily understood descriptions:
filtered_df['Weather'].replace('Unk', 'UNK', inplace=True)
filtered_df['Weather'].replace(['VMC', 'IMC', 'UNK'] , 
                                         ['Decent', 'Poor', 'Unknown'], inplace=True)

# Then, we need to replace the null values with 'Unknown':
filtered_df['Weather'].fillna('Unknown', inplace = True)

filtered_df['Weather'].value_counts()

Decent     18981
Unknown     2818
Poor        1436
Name: Weather, dtype: int64

In [16]:
# One more thing: we'll create a boolean column to indicate whether or not the accident involved a fatal injury.
# This way, we can use the 'Sum' function for this column in Tableau to count the total number of accidents by manufacturer:
filtered_df['Fatal Injury Check'] = np.where(filtered_df['Total.Fatal.Injuries'] == 0, 0, 1)

filtered_df.head()

,Event.Date,Location,Make,Purpose,Total.Fatal.Injuries,Weather,Fatal Injury Check
47675,2000-01-01,United States,Cessna,Commercial & Public,0.0,Decent,0
47677,2000-01-02,United States,Cessna,Personal,0.0,Decent,0
47678,2000-01-02,United States,Cessna,Personal,0.0,Decent,0
47679,2000-01-02,United States,Piper,Personal,0.0,Decent,0
47680,2000-01-02,United States,Piper,Personal,0.0,Decent,0


In [17]:
# Finally, the filtered data is written to Excel to visualize in Tableau:
filtered_df.to_excel('filtered_aircraft_data.xlsx')

### Data Analysis and Conclusions
For the three business questions, I prepared visualizations on Tableau. Link to the Dashboard: https://public.tableau.com/app/profile/yi.wei.liu/viz/Phase1Project_16984566959940/AnalysisDashboard

In short, Boeing is overall the manufacturer with the best safety record. Only 28% (461 of 1,675) of Boeing's accidents involved a fatal injury, vs. 50% (10,860 of 21,560) for all manufacturers ex-Boeing.

#### 1) Location
- Across all manufacturers, 48% of U.S. flight accidents involved a fatal injury vs. 52% of overseas flights. 
- Within U.S. flights, 39% (233 of 595) of Boeing's accidents involved a fatal injury, vs. 48% (9,408 of 19,423) for all manufacturers ex-Boeing.
- Within overseas flights, it was 21% (228 of 1,080) for Boeing vs. 68% (1,452 of 2,137) for others.

Recommendation:
- There is only a slightly higher safety risk overseas compared to the U.S. 
- If the Company decides it is commercially feasible to operate overseas, I recommend Boeing as the only manufacturer safe enough for overseas flights. For domestic flights, Boeing is still the best choice, but other manufacturers such as Robinson and Air Tractor are not far behind. 
- An avenue for further inquiry is why the fatal injury ratio for Boeing is much lower overseas than in the U.S. (21% vs 39%).

#### 2) Purpose
- Across all manufacturers, 48% (3,389 of 7,057) of commercial aircraft accidents involved a fatal injury vs. 49% for personal aircraft (7,932 of 16,178).
- Within commercial flights, 27% (418 of 1,545) of Boeing's accidents involved a fatal injury vs. 54% (2,971 of 5,512) for all manufacturers ex-Boeing.
- Within personal flights, it was 33% (43 of 130) for Boeing vs. 49% (7,889 of 16,048) for others.

Recommendation:
- The safety risk for commercial and personal aircraft are similar. The Company may operate both types of aircraft depending on commercial feasibility. Boeing is the safest option for both.

#### 3) Weather
- Across all 23,235 accidents, 82% involved decent weather and 6% involved poor weather; weather for 12% of accidents were unknown.
- Across 11,321 accidents with a fatal injury, 80% involved decent weather and 10% involved poor weather; weather for 10% were unknown.
- In decent weather, 48% of all accidents involved a fatal injury. In poor weather, 81% of accidents involved a fatal injury.

Recommendation:
- In poor weather, the fatal injury rate was significantly higher than average. I would recommend our company not to operate aircraft in poor weather to reduce the risk of fatal injuries.

Thank you for reading this notebook!